data from http://jmcauley.ucsd.edu/data/tradesy/
```
wget http://jmcauley.ucsd.edu/data/tradesy/tradesy.json.gz
wget http://jmcauley.ucsd.edu/data/tradesy/tradesy_item_urls.json.gz
```


paper: VBPR: Visual Bayesian Personalized Ranking from Implicit Feedback  
   https://arxiv.org/abs/1510.01784

In [4]:
import tensorflow as tf
import os
import pickle
import random
import gzip
import struct
from collections import defaultdict
import gensim as gs
import numpy as np
try:
    # noinspection PyUnresolvedReferences
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print("notebook")
        from tqdm import tqdm_notebook as tqdm
    else:
        raise RuntimeError
except (NameError, RuntimeError):
    from tqdm import tqdm

notebook


In [5]:
user_data = eval(gzip.open("./tradesy.json.gz", 'r').read())

In [6]:
def load_data(user_data, max_uid=1000000):
    line_list =[]
    user_list = []
    item_dic = {}
    few_buyers =[]
    data = defaultdict(set)
    for d  in user_data:
        user_id = d["uid"]
        items = d["lists"]["bought"]
        
        item_list = [int(i) for i in items]
        if item_list:
            max_i = max(item_list)
            if max_i < max_uid:
                user_list.append(user_id)
                line_list.append(items)
            
    dictionary = gs.corpora.Dictionary(line_list)
    dictionary.filter_extremes(no_below=1)
    dictionary.compactify()
    for u, items in zip(user_list, line_list):
        data[u].update([dictionary.token2id[item] for item in items if item in dictionary.token2id])
    for u,i in data.items():
        if len(i) < 5:   # 5 same as the paper's
            few_buyers.append(u)
    for u in few_buyers:
        del data[u]
    d = {}
    user_list = []
    for idx,(u,i) in enumerate(data.items()):
        d[idx] = i
        user_list.append(u)
    user_count = len(data.keys())
    item_count = len(dictionary)
    return user_count, item_count, d, dictionary, user_list

In [7]:
def generate_test(data):
    user_test = dict()
    for u, i_list in data.items():
        if i_list:
            user_test[u] = np.random.choice(list(i_list))
    return user_test

In [8]:
max_uid=1000000
user_count, item_count, data, dictionary,u_list  = load_data(user_data, max_uid)
print("item count: ", item_count)
print("user count: ", user_count)
ui_test = generate_test(data)

item count:  32807
user count:  1076


In [9]:
#file_name = "./items.pickle"
#with open(file_name ,mode='wb') as f:
#    pickle.dump(list(dictionary.token2id.keys()), f, protocol=4)

In [10]:
def readImageFeatures(path, dictionary):
    f = open(path, 'rb')
    imgs = {}
    uids = dictionary.token2id.keys()
    count = 0
    while f:
        userId = f.read(10)
        userId = userId.strip()
        if userId == '':
            break
        uid =  userId.decode('ascii')
        if uid in uids:
            feature = [struct.unpack('f', f.read(4)) for _ in range(4096)]
            imgs[dictionary.token2id[uid]] = feature
            count += 1
            if count == len(uids):
                break
        else:
            f.read(4*4096)
            
    file_name = "./images.pickle"
    with open(file_name ,mode='wb') as f:
        pickle.dump(image_features, f, protocol=4)
    return imgs

In [11]:
#image_features = readImageFeatures("./image_features_tradesy.b", dictionary)

In [12]:
file_name = "./images.pickle"
with open(file_name, mode='rb') as f:
          image_features =  pickle.load(f)

In [13]:
len(image_features)

32807

In [14]:
len(image_features[13348])

4096

In [15]:
def uniform_sample_batch(train_ratings, item_count, image_features, sample_count=20000, batch_size=5):
    for i in range(sample_count):
        t = []
        iv = []
        jv = []
        for b in range(batch_size):
            u = random.sample(train_ratings.keys(), 1)[0]
            i = random.sample(train_ratings[u], 1)[0]
            j = random.randint(0, item_count-1)
            while j in train_ratings[u]:
                j = random.randint(0, item_count-1)
            t.append([u, i, j])
            iv.append(image_features[i])
            jv.append(image_features[j])
        yield np.asarray(t), np.hstack(tuple(iv)), np.hstack(tuple(jv))

def test_batch_generator_by_user(train_ratings, test_ratings, item_count, image_features, n_user=10):  
    for u in np.random.choice(list(test_ratings.keys()), n_user):
        i = test_ratings[u]
        t = []
        ilist = []
        jlist = []
        for j in range(item_count):
            if j != test_ratings[u] and not (j in train_ratings[u]):
                t.append([u, i, j])
                ilist.append(image_features[i])
                jlist.append(image_features[j])
        yield np.asarray(t), np.hstack(tuple(ilist)), np.hstack(tuple(jlist))

In [16]:
def weight_variable(shape):
    return tf.Variable(tf.random_normal(shape, mean=0.0, stddev=0.01))

def bias_variable(shape):
    return tf.Variable(tf.random_normal(shape, mean=0.0, stddev=0.01))

In [25]:
def vbpr(user_count, item_count, latent_dim=20, latent_img_dim=128, learning_rate = 0.001,l2_regulization = 1.0):
    image_dim = 4096
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])
    iv = tf.placeholder(tf.float32, [4096, None])
    jv = tf.placeholder(tf.float32, [4096, None])
    
    with tf.device("/cpu:0"):
        
        user_w = weight_variable([user_count+1, latent_dim])
        user_img_w = weight_variable([user_count+1, latent_img_dim])
        item_w = weight_variable([item_count+1, latent_dim])
        item_b = bias_variable([item_count+1, 1])
        
        u_e = tf.nn.embedding_lookup(user_w, u)
        u_img = tf.nn.embedding_lookup(user_img_w, u)
        
        i_e = tf.nn.embedding_lookup(item_w, i)
        i_b = tf.nn.embedding_lookup(item_b, i)
        j_e = tf.nn.embedding_lookup(item_w, j)
        j_b = tf.nn.embedding_lookup(item_b, j)
    
    with tf.device("/cpu:0"):
   
        img_w = weight_variable([4096, latent_img_dim])

        img_i_j = tf.matmul(tf.transpose(iv - jv), img_w)

        x = i_b - j_b + tf.reduce_sum(tf.matmul(u_e, tf.transpose(i_e - j_e)), 1, keep_dims=True) +\
            tf.reduce_sum(tf.matmul(u_img, tf.transpose(img_i_j)),1, keep_dims=True)

        auc = tf.reduce_mean(tf.to_float(x > 0))

        l2_norm = tf.add_n([
                tf.reduce_sum(tf.norm(u_e)), 
                tf.reduce_sum(tf.norm(u_img)),
                tf.reduce_sum(tf.norm(i_e)),
                tf.reduce_sum(tf.norm(j_e)),
                tf.reduce_sum(tf.norm(img_w)),
                tf.reduce_sum(tf.norm(i_b)),
                tf.reduce_sum(tf.norm(j_b))
            ])

        loss = l2_regulization * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return u, i, j, iv, jv, loss, auc, train_op

In [26]:
test_n_user = 30 #len(ui_test)
sample_count = 500
n_epoch = 5000


In [ ]:
with tf.Graph().as_default(), tf.Session() as session:
    with tf.variable_scope('vbpr'):
        u, i, j, iv, jv, loss, auc, train_op = vbpr(user_count, item_count,learning_rate = 0.0001)
    
    session.run(tf.global_variables_initializer())
    for epoch in tqdm(range(n_epoch)):
        
        _loss_train = 0.0
        for d,i_img,j_img  in uniform_sample_batch(data, item_count, image_features, sample_count=sample_count):
            _loss, _ = session.run([loss, train_op], feed_dict={
                    u:d[:,0], i:d[:,1], j:d[:,2], iv: i_img, jv: j_img
                })
            _loss_train += _loss
            
        print("epoch ", epoch, " train_loss:", _loss_train/sample_count)
        
        if epoch % 10 == 0 and epoch != 0:
            _auc_all = 0.0
            _loss_test = 0.0
            for d,i_img,j_img in tqdm(test_batch_generator_by_user(data, ui_test, item_count, image_features, n_user=test_n_user)):
                _loss, _auc = session.run([loss, auc], feed_dict={
                        u:d[:,0], i:d[:,1], j:d[:,2], iv: i_img, jv: j_img
                })
                _loss_test += _loss
                _auc_all += _auc
            print( "test_loss: ", _loss_test/test_n_user, " auc: ", _auc_all/test_n_user)


epoch  0  train_loss: 4.53547286701
epoch  1  train_loss: 1.4604648037
epoch  2  train_loss: 1.2721886456
epoch  3  train_loss: 1.19367926931
epoch  4  train_loss: 1.11112071371
epoch  5  train_loss: 1.03883358324
epoch  6  train_loss: 0.972820601225
epoch  7  train_loss: 0.927639446139
epoch  8  train_loss: 0.900886861086
epoch  9  train_loss: 0.887057055473
epoch  10  train_loss: 0.875901062369
test_loss:  14.3292363803  auc:  0.41418132782
epoch  11  train_loss: 0.867774270415
